<a href="https://colab.research.google.com/github/Saim-Hassan786/Learn-Agentic-AI/blob/main/Runner-Class/Runner_Class_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runner
**Runner is very crucial and imperative feature of OpenAI Agents SDK , it acts as an Agent loop that is responsible for orchestrating ,running and implementing all the Agents,Tools,Guardrails,Handoffs,LLM configuration etc until the desired ouput in relative to the user query is achieved.It can be regarded as a manager loop that keeps running with all the sdk code logic until it achieves the desired output or encounter an error.**

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.7 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('GOOGLE_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,set_tracing_disabled
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key = GOOGLE_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

# Runner Simple Example

In [ ]:
from agents import Agent,Runner

result = Runner.run_sync(
    starting_agent = Agent(
    name = "Assistant Agent",
    model = "gemini-2.5-flash"
),
    input = "what is the capital of sweden"
)
result.final_output

'The capital of Sweden is **Stockholm**.'

# Runner Class Methods
A runner class has 3 cls methods :

1. **Runner.run()** :: asynchronous
2. **Runner.run_sync()** :: synchronous block over async Runner.run
3. **Runner.run_streamed()** :: async generator of stream events for Streming

# Runner.run()
Runner.run is an async method that has following parameters and return a **RunResult** Object

1. **starting_agent** :: positional argument
2. **input** :: positional argument
3. **context** :: keyword argument
4. **hooks** :: RunHooks :: keyword argument
5. **run_config** :: RunConfig :: keyword argument
6. **max_turns** :: DEFAULT_MAX_TURNS
6. **previous_response_id** :: For Responses APIs

In [ ]:
from agents import Agent,Runner,function_tool
from pydantic import BaseModel

class Input(BaseModel):
    name : str
    age : int

@function_tool
def add(a:int,b:int):
    return a+b

agent_1 = Agent(
    name = "Assistant Agent",
    model = "gemini-2.5-flash",
    tools = [add],
    instructions="HelpFul Assistant",
    tool_use_behavior="stop_on_first_tool"
    # tool_use_behavior = "run_llm_again"
)

result_1 = await Runner.run(
    starting_agent = agent_1,
    input = "what is the answer of adding 4 and 5",
    max_turns = 1,
    # max_turn = 2  It will generate an error as the LLM asks for tool call and toll result when goes back to LLM the Max Turns will be exceeded and it errors
)
print(result_1.final_output)

# 2nd case
agent_2 = Agent(
    name = "Assistant Agent",
    model = "gemini-2.5-flash",
    instructions="General purpose Assistant that answer the user queries",
)

result_2 = await Runner.run(
    starting_agent = agent_2,
    input = "what is the capital of france",
    max_turns = 1, # As LLM respond with the ouput that is considered as the FinalOutput so the loop terminates with answer and result is achieved in 1 max turn without error
)
print(result_2.final_output)

# 3rd case
agent_3 = Agent(
    name = "Assistant Agent",
    model = "gemini-2.5-flash",
    tools = [add],
    instructions="General purpose Assistant that answer the user queries and use tool when required",
)

result_3 = await Runner.run(
    starting_agent = agent_3,
    input = "what is my name",
    max_turns = 7,
    context = Input(name = "Saim",age = 25) #context is only available for Agent,Tools etc internally ,never given to LLM
)
print(result_3.final_output)


9
The capital of France is **Paris**.
I do not know your name. I am a large language model, not a personal assistant.



# OverAll Working Of Runner.run()

1. When we start an agent loop with Runner.run(), according to new experimental documentation, DEFAULT_AGENT_RUNNER class is initialized and then all the parameters are called with the **run** method in it.
2. In run() method, first all th parameters are retrieved and 2 parameters are initialized if not given **run_config** and **hooks**.
3. Then the next preparations are as follows:

    - **tool-use_tracker** is initialized
    - **should_start_agent_hook** is initialized to True
    - **Tracing** is enabled from all the parameters of run_config
    - **current_span** is initialized
    - **generated_items** list is [ ] initialized
    - **model_response** list is [ ] initialized
    - **context** is wrapped in RuncontextWrapper
    - **input_quardrails** list is [ ] initialized
    - **output_quardrails** list is [ ] initialized
    - **original_input** deepcopy is initialized
    - **current_turn** is set to 0
    - **current_agent** is set to **starting_agent** passed in Runner.

4. After all the above preparations a **current_span** is initialzed if it is None with all the **handoffs**, **output_schema**, **tools** and **name** of the current_agent in a **while** loop
5. Now if the **current_turn** is greater than **max_turns** defined in Runner.run() then the Error is spaned in current_span and the **MaxTurnsExceeded** error is raised and the loop terminates
6. If **current_turn** is equal to 1 then all the **input_guardrails** are retrived from the current_agent and run_config are run with the original input.If no exception is raised then the **run_single_turn** func with the above retreived parameters are run.
7. An **else** block is also set to execute the **run_single_turn** func if the current turn is more that 1 and thi is run without **input_gaurdrails**.
8. After the 1st turn, all the generated_items,model_responses and original_items are appended to the **turn_result** variable that is initialized for running **run_single_turn**
8. Now after the 1st turn, LLM decides what will be the **Next_Step**
  
   - If next_step is **NextStepFinalOutput** then all the **output_gaurdrails** are retrived from the current_agent and run_config and are run with the **output** and the if no error then loop terminates with **RunResult** containing final output.
   - If next_step is **NextStepHandOff** then the **current_agent** is changed with the **new_agent** name, the **current_span** is finished, **current_span** is set to None and **should_start_agent_hook** is also set to True for the new_agent to take control.
   - If next_step is **NextStepRunAgain** then just use **pass** for the loop to continue the next turn.
   - If there is any next_step that is undefined then **AgentExceptionError** is rasied with all the error details
10. At last , is there is any **current_span** running , finishes it off gracefully.



In [ ]:
# Runner.run() example
from agents import Agent,Runner

agent_with_run = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    model = "gemini-2.5-flash",
)
result_with_run = await Runner.run(
    starting_agent = agent_with_run,
    input = "who is Geoffery Hintman"
)
print(result_with_run.final_output)

**Geoffrey Hinton** is a highly renowned British-Canadian cognitive psychologist and computer scientist, widely considered one of the "Godfathers of AI" or the "Godfather of Deep Learning."

Here's why he's so important:

1.  **Pioneering Deep Learning:** Hinton is a central figure in the field of artificial intelligence, particularly for his groundbreaking work on artificial neural networks and deep learning. His research laid much of the theoretical and practical foundation for the deep learning revolution we're seeing today.

2.  **Backpropagation Algorithm:** One of his most significant contributions was his work on the **backpropagation algorithm** in the 1980s. This algorithm was crucial for efficiently training multi-layer neural networks, making them practical for real-world applications.

3.  **Neural Network Research:** He dedicated decades to exploring how neural networks learn, represent information, and perform complex tasks like speech recognition and computer vision, lon

# OverAll Working Of Runner.run_sync

1. Similary, When we start an agent loop with Runner.run_sync(), according to new experimental documentation, DEFAULT_AGENT_RUNNER class is initialized and then all the parameters are called with the **run_sync** method in it.
2. It in the same way retrives all the parameters passed to it and others not passsed are set to None.
3. It is just a synchronous block over the asynchronous **run()** method.
4. So what it simple does here is it gets a new event_loop and execute the **run()** method with all the parameters retreived passed in it and **block its execution until completion** to make it a synchronous function at the end.
5. So async **run()** executes in it until completion and at the end the **RunResult** object is returned with teh final output.

In [ ]:
from agents import Agent, Runner

agent_with_run_sync = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    model = "gemini-2.5-flash",
)

# import nest_asyncio  In Jupyter nootbooks as the event loop is already running so creating a new event loop is not permitted by jupyter to run our run_sync method we have to run this code that enables the nested loop execution.
# nest_asyncio.apply()

result_with_run_sync = Runner.run_sync(
    starting_agent = agent_with_run_sync,
    input = "who is the founder of OpenAI"
)
print(result_with_run_sync.final_output)

OpenAI was founded by a group of individuals, so there isn't one single "founder" in the traditional sense. It was established as a non-profit research company in **December 2015** by:

*   **Sam Altman**
*   **Elon Musk**
*   **Ilya Sutskever**
*   **Greg Brockman**
*   **Wojciech Zaremba**
*   **John Schulman**

Elon Musk was an initial co-chairman and significant funder, though he later stepped down from the board in 2018. Sam Altman has served as the CEO for most of its operational history.


# OverAll Working Of Runner.run_streamed()

1. Similary, When we start an agent loop with Runner.run_streamed(), according to new experimental documentation, DEFAULT_AGENT_RUNNER class is initialized and then all the parameters are called with the run_streamed method in it.
2. All the parameters are retrieved and passed in run_streamed(), then a variable named **streamed_result** is initialzied for a **RunResultStreaming** class and filled with paramters like, current_agent, current_span, is_complete, input_guardrails_list,output_guardrails_list,hooks , run_config , started trace, wrapping context in RunContextWrapper etc we have done it to return the RunResultStreaming object immediately and we will keep updating with the real values created by the **start_streaming** function below.
3. Then a couroutine is created and the function named **start_streaming** is run with all the actual parameters passed in **run_streamed** with an additional **streamed_result** passed in it as well that is the instance of **RunResultStreaming**.
4. In **start_streaming** it starts with all the original parameters passed in **run_streamed** with an additional **RunResultStreaming** object for updating continously in real time. In it fist all the parameters like **current_span**, **tool_use_tracker**, **should_start_agent_hook** , **current_turn** etc and an **AgentUpdatedStreamedEvent** is sent to **stream_event_queue** to stream.
5. A while loop is started and if **streamed_result.is_complete** is Ture then break the loop and streaming ,then a **current_span** is created with all the tools, handoffs,output_schema and current_agent name and the current_turn counter is set to **+= 1** after each turn.
6. Similarly , if **current_turn** is greature than **max_turns** then the **QueueSentinel** is rasied in **streamed_result** event queue and the loop is broken.
7. If the **current_turn** is **== 1** then run the **run_input_guardrails_with_queue** on all the guardrails from the current_agent and the run_config guardrails as well and if no error run the **run_single_turn_streamed** with all the parameters and **should_start_agent_hook** is set to False after first turn.
8. And an **else** bolck is also set there if the current_turn is more than 1 to run **run_single_turn-streamed** but this time without **run_input_guardrails_with_queue** and the **raw_responses**, **new_items**, **original_input** are updated in the streamed_result continously for streaming.
9. Then the LLM decide what will be the next_step:

   - If next_step is **NextStepFinalOutput** then all the **output_gaurdrails** are retrived from the current_agent and run_config and are run with the **output** and the if no error then all the events are updated in the streamed_result and **QueueSentinel** is sent to signal no upcoming_streames and the **is_complete** is set to true that terminates the loop..
   - If next_step is **NextStepHandOff** then the **current_agent** is changed with the **new_agent** name,**AgentUpdatedStreamEvent** is sent to signal new agent taking control, the **current_span** is finished, **current_span** is set to None and **should_start_agent_hook** is also set to True .
   - If next_step is **NextStepRunAgain** then just use **pass** for the loop to continue the next turn.
   - If there is any next_step that is undefined then **AgentException** **is_complete** is set to True that breakes the loop, **QueueSentinel** is sent to signal stopping stream events and the it is raised containing all the Error details

10. At last , is there is any **current_span** running , finishes it off gracefully.

In [ ]:
from agents import Agent,Runner
from openai.types.responses import ResponseTextDeltaEvent

agent_with_run_streamed = Agent(
    name="Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    model = "gemini-2.5-flash"
)

result_with_run_streamed = Runner.run_streamed(
    starting_agent = agent_with_run_streamed,
    input = "who is the founder of Anthropic"
)
async for event in result_with_run_streamed.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Anthropic doesn't have a single "founder" in the traditional sense, as it was started by a group of former OpenAI employees.

However, the most prominent figures and the core of the founding team are:

*   **Dario Amodei** (CEO)
*   **Daniela Amodei** (President)

They, along with many other researchers, left OpenAI primarily due to disagreements over the direction and safety focus of AI development, and then co-founded Anthropic in 2021 to focus on large language models and AI safety research in a more controlled and responsible way.

# RunResult
**The Return Objects from Runner.run() and Runner.run_sync()**

In [ ]:
from agents import Agent,Runner

agent_with_run_result = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    model = "gemini-2.5-flash",
)
result_with_run_result = await Runner.run(
    starting_agent = agent_with_run_result,
    input = "what is the capital of Germany"
)

In [ ]:
print(result_with_run_result)
print("=========="*20)
print(result_with_run_result.final_output)
print("=========="*20)
print(result_with_run_result.new_items)
print("=========="*20)
print(result_with_run_result.raw_responses)
print("=========="*20)
print(result_with_run_result.input_guardrail_results)
print("=========="*20)
print(result_with_run_result.output_guardrail_results)
print("=========="*20)
print(result_with_run_result.last_agent)
print("=========="*20)
print(result_with_run_result.last_response_id)
print("=========="*20)
print(result_with_run_result.context_wrapper)
print("=========="*20)

RunResult:
- Last agent: Agent(name="Assistant Agent", ...)
- Final output (str):
    The capital of Germany is **Berlin**.
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)
The capital of Germany is **Berlin**.
[MessageOutputItem(agent=Agent(name='Assistant Agent', instructions='General purpose Assistant that answer the user queries', prompt=None, handoff_description=None, handoffs=[], model='gemini-2.5-flash', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, response_include=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, hooks=None, tool_use_behavior='run_llm_again', reset_tool

# RunResultStreaming
**The Return Object from Runner.run_streamed()**

In [ ]:
from agents import Agent,Runner
from openai.types.responses import ResponseTextDeltaEvent

agent_with_run_result_streaming = Agent(
    name="Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    model = "gemini-2.5-flash"
)

result_with_run_result_streaming = Runner.run_streamed(
    starting_agent = agent_with_run_streamed,
    input = "who is the prime minister of pakistan"
)
async for event in result_with_run_result_streaming.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

The current Prime Minister of Pakistan is **Shehbaz Sharif**.

He assumed office for his second term on March 4, 2024, following the general elections held in February 2024.

In [ ]:
print(result_with_run_result_streaming)
print("=========="*20)
print(result_with_run_result_streaming.final_output)
print("=========="*20)
print(result_with_run_result_streaming.new_items)
print("=========="*20)
print(result_with_run_result_streaming.raw_responses)
print("=========="*20)
print(result_with_run_result_streaming.input_guardrail_results)
print("=========="*20)
print(result_with_run_result_streaming.output_guardrail_results)
print("=========="*20)
print(result_with_run_result_streaming.last_agent)
print("=========="*20)
print(result_with_run_result_streaming.last_response_id)
print("=========="*20)
print(result_with_run_result_streaming.context_wrapper)
print("=========="*20)
print(result_with_run_result_streaming.is_complete)
print("=========="*20)
print(result_with_run_result_streaming.current_turn)
print("=========="*20)
print(result_with_run_result_streaming.max_turns)
print("=========="*20)
print(result_with_run_result_streaming.current_agent)
print("=========="*20)

RunResultStreaming:
- Current agent: Agent(name="Assistant Agent", ...)
- Current turn: 1
- Max turns: 10
- Is complete: True
- Final output (str):
    The current Prime Minister of Pakistan is **Shehbaz Sharif**.
    
    He assumed office for his second term on March 4, 2024, following the general elections held in February 2024.
- 1 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResultStreaming` for more details)
The current Prime Minister of Pakistan is **Shehbaz Sharif**.

He assumed office for his second term on March 4, 2024, following the general elections held in February 2024.
[MessageOutputItem(agent=Agent(name='Assistant Agent', instructions='General purpose Assistant that answer the user queries', prompt=None, handoff_description=None, handoffs=[], model='gemini-2.5-flash', model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=Non

# RunConfig
**The configuration settings in the run_config parameter of Runner that will override the settings in model_settings parameter of Agent class.**

# Below is the Example Of :

**1. model**

**2. model_provider**

**3. model_settings**

In [ ]:
from agents import Agent,Runner,RunConfig,ModelSettings,ModelProvider,OpenAIChatCompletionsModel,Model

agent_with_run_config = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
)

class CustomModelProvider(ModelProvider):
  def get_model(self,model_name)->Model:
    return OpenAIChatCompletionsModel(model = model_name, openai_client=external_client)

result_with_run_config = await Runner.run(
    starting_agent = agent_with_run_config,
    input = "Who is the founder of Python language and why it is named so",
    run_config= RunConfig(
        model = "gemini-2.5-flash",
        model_provider = CustomModelProvider(),
        model_settings=ModelSettings(
            temperature = 0.1,
            max_tokens = 1000
        )
    )
)

In [ ]:
print(result_with_run_config.final_output)

The founder of the Python language is **Guido van Rossum**.

He is a Dutch programmer who began working on Python in the late 1980s, specifically around December 1989, at Centrum Wiskunde & Informatica (CWI) in the Netherlands. He served as the project's Benevolent Dictator For Life (BDFL) until July 2018, meaning he oversaw the Python development process and made final decisions.

As for **why it is named Python**:

It's not named after the snake, as many might assume! Guido van Rossum was a big fan of the British comedy sketch show **"Monty Python's Flying Circus"**.

He wanted a name that


# handoff_input_filter

In [ ]:
from agents import Agent,Runner,RunConfig,ModelSettings,ModelProvider,OpenAIChatCompletionsModel,Model,HandoffInputData

agent_with_run_config_handoff = Agent(
    name = "Essay Writer Agent",
    instructions="Write a 50 words essay for the topic given to you",
)

agent_with_run_config_2 = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    handoffs=[agent_with_run_config_handoff]
)

def my_handoff_filter(data: HandoffInputData) -> HandoffInputData:
    print("📦 HandoffInputFilter triggered!")
    print("🔹 Original input_history:", data.input_history)

    new_input = data.input_history
    return HandoffInputData(
        input_history=(new_input),
        pre_handoff_items=data.pre_handoff_items,
        new_items=data.new_items,
    )

class CustomModelProvider(ModelProvider):
  def get_model(self,model_name)->Model:
    return OpenAIChatCompletionsModel(model = model_name, openai_client=external_client)

result_with_run_config_2 = await Runner.run(
    starting_agent = agent_with_run_config_2,
    input = "Write an essay about 'Meta AI'",
    run_config= RunConfig(
        model = "gemini-2.5-flash",
        model_provider = CustomModelProvider(),
        model_settings=ModelSettings(
            temperature = 0.1,
            max_tokens = 1000
        ),
        handoff_input_filter = my_handoff_filter,
    )
)
print(result_with_run_config_2.final_output)

📦 HandoffInputFilter triggered!
🔹 Original input_history: Write an essay about 'Meta AI'
Meta AI powers intelligent features across Facebook, Instagram, and WhatsApp, enhancing user experiences with personalized content and creative tools. It also drives cutting-edge research in large language models and computer vision, shaping AI's future and the metaverse. This ongoing development aims to create more intuitive and immersive digital interactions for billions worldwide.


# input_guardrails

In [ ]:
from agents import Agent,Runner,RunConfig,ModelSettings,ModelProvider,OpenAIChatCompletionsModel,Model,GuardrailFunctionOutput,InputGuardrailTripwireTriggered,HandoffInputData,InputGuardrail,RunContextWrapper
from pydantic import BaseModel

class Filter(BaseModel):
  reasoning : str
  is_essay_asked : bool

check_essay_gaurdrail_agent = Agent(
     name = " Guardrail Agent",
     instructions="Check whether the user asked about writing an essay",
     output_type= Filter,
     model = "gemini-2.5-flash"
 )

def input_quardrail(ctx: RunContextWrapper,agent:Agent, input_data)->GuardrailFunctionOutput:
  print("Input Guardrail Working")
  result = Runner.run_sync(
      check_essay_gaurdrail_agent,
      input = input_data,
      context = ctx.context
  )
  final_result = result.final_output_as(Filter)
  return GuardrailFunctionOutput(
      output_info = final_result,
      tripwire_triggered = not final_result.is_essay_asked
  )

agent_with_run_config_handoff = Agent(
    name = "Essay Writer Agent",
    instructions="Write a 50 words essay for the topic given to you",
)

agent_with_run_config_2 = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    handoffs=[agent_with_run_config_handoff]
)

def my_handoff_filter(data: HandoffInputData) -> HandoffInputData:
    print("📦 HandoffInputFilter triggered!")
    print("🔹 Original input_history:", data.input_history)

    new_input = data.input_history
    return HandoffInputData(
        input_history=(new_input),
        pre_handoff_items=data.pre_handoff_items,
        new_items=data.new_items,
    )

class CustomModelProvider(ModelProvider):
  def get_model(self,model_name)->Model:
    return OpenAIChatCompletionsModel(model = model_name, openai_client=external_client)

try :
  result_with_run_config_2 = await Runner.run(
      starting_agent = agent_with_run_config_2,
      input = "What is the capital of Italy",
      run_config= RunConfig(
          model = "gemini-2.5-flash",
          model_provider = CustomModelProvider(),
          model_settings=ModelSettings(
              temperature = 0.1,
              max_tokens = 1000
          ),
          handoff_input_filter = my_handoff_filter,
          input_guardrails = [InputGuardrail(
              guardrail_function = input_quardrail
          )]

      )
  )
except InputGuardrailTripwireTriggered as e:
  print(e)
  print(e.guardrail_result.output.output_info.reasoning)
  print(e.guardrail_result.output.output_info.is_essay_asked)

print(result_with_run_config_2.final_output)

Input Guardrail Working
Guardrail InputGuardrail triggered tripwire
The user is asking a factual question about the capital of Italy, not about writing an essay.
False



# output_guradrails

In [ ]:
from typing_extensions import final
from agents import Agent,Runner,RunConfig,ModelSettings,ModelProvider,OpenAIChatCompletionsModel,Model,GuardrailFunctionOutput,OutputGuardrail,OutputGuardrailTripwireTriggered,InputGuardrailTripwireTriggered,HandoffInputData,InputGuardrail,RunContextWrapper
from pydantic import BaseModel

class Filter(BaseModel):
  reasoning : str
  is_essay_asked : bool

check_essay_gaurdrail_agent = Agent(
     name = " Guardrail Agent",
     instructions="Check whether the user asked about writing an essay",
     output_type= Filter,
     model = "gemini-2.5-flash"
 )

def input_quardrail(ctx: RunContextWrapper,agent:Agent, input_data)->GuardrailFunctionOutput:
  print("Input Guardrail Working")
  result = Runner.run_sync(
      check_essay_gaurdrail_agent,
      input = input_data,
      context = ctx.context
  )
  final_result = result.final_output_as(Filter)
  return GuardrailFunctionOutput(
      output_info = final_result,
      tripwire_triggered = not final_result.is_essay_asked
  )

class Output_Filter(BaseModel):
  response : str
  is_response_contains_name : bool

check_name_output_guardrail_agent = Agent(
    name = "Output Guardrail Agent",
    instructions="Check whether the output contains the name",
    output_type= Output_Filter,
    model = "gemini-2.5-flash"
)

def output_quardrail(ctx: RunContextWrapper,agent:Agent, input_data)->GuardrailFunctionOutput:
  print("Output Guardrail Working")
  result = Runner.run_sync(
      check_name_output_guardrail_agent,
      input = input_data,
      context = ctx.context
  )
  final_result = result.final_output_as(Output_Filter)
  return GuardrailFunctionOutput(
      output_info = final_result,
      tripwire_triggered = final_result.is_response_contains_name
  )

agent_with_run_config_handoff = Agent(
    name = "Essay Writer Agent",
    instructions="Write a 50 words essay for the topic given to you",
)

agent_with_run_config_2 = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    handoffs=[agent_with_run_config_handoff]
)

def my_handoff_filter(data: HandoffInputData) -> HandoffInputData:
    print("📦 HandoffInputFilter triggered!")
    print("🔹 Original input_history:", data.input_history)

    new_input = data.input_history
    return HandoffInputData(
        input_history=(new_input),
        pre_handoff_items=data.pre_handoff_items,
        new_items=data.new_items,
    )

class CustomModelProvider(ModelProvider):
  def get_model(self,model_name)->Model:
    return OpenAIChatCompletionsModel(model = model_name, openai_client=external_client)

try :
  result_with_run_config_2 = await Runner.run(
      starting_agent = agent_with_run_config_2,
      input = "Write an essay about me with my name, I am an AI Engineer and my name is 'Saim'",
      run_config= RunConfig(
          model = "gemini-2.5-flash",
          model_provider = CustomModelProvider(),
          model_settings=ModelSettings(
              temperature = 0.1,
              max_tokens = 1000
          ),
          handoff_input_filter = my_handoff_filter,
          input_guardrails = [InputGuardrail(
              guardrail_function = input_quardrail
          )],
          output_guardrails = [OutputGuardrail(
              guardrail_function = output_quardrail
          )]

      )
  )
except (InputGuardrailTripwireTriggered,OutputGuardrailTripwireTriggered) as e:
  print(e)
  print(e.guardrail_result.output.output_info)
  print(e.guardrail_result.output.tripwire_triggered)

print(result_with_run_config_2.final_output)

Input Guardrail Working
📦 HandoffInputFilter triggered!
🔹 Original input_history: Write an essay about me with my name, I am an AI Engineer and my name is 'Saim'
Output Guardrail Working
Guardrail OutputGuardrail triggered tripwire
response="Saim, an AI Engineer, is a visionary architect of the future. He meticulously designs and implements intelligent systems, transforming complex data into innovative solutions. His expertise empowers machines to learn, think, and assist, pushing the boundaries of what's possible. Saim's dedication to artificial intelligence isn't just a career; it's a passion for building a smarter, more connected world." is_response_contains_name=True
True



# Other Parameters Include :

**1. tracing_disabled**

**2. trace_id**

**3. group_id**

**4. workflow_name**

**5. tracing_include_sensitive_data**

**6. tracing_metadata**

In [ ]:
from agents import Agent,Runner,RunConfig

agent_with_run_config_3 = Agent(
    name = "Assistant Agent",
    instructions="General purpose Assistant that answer the user queries",
    model = "gemini-2.5-flash"
)

result = await Runner.run(
    starting_agent = agent_with_run_config_3,
    input = "who is the founder of Anthropic",
    run_config=RunConfig(
        tracing_disabled=True,   # tracing_disabled means no tracing is donwe for the agent run like no logging
        trace_include_sensitive_data=False,   # It means that if tracing enabled it does not include any sensitive data from context or tool calls
        workflow_name="Agentic WorkFlow",   # name of our workflow for better understanding and distinguishing from other workflows
        trace_id="my_trace-786",             # unique trace_id for every trave if not given SDk creates by it self
        group_id="my_chat_group_thread_786",  # to connect multiple traces togather like a thread
        trace_metadata={
            "my_key": "my_value"             # additional data to include in our tracing
        }
    )
)